In [26]:
import csv
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pulp import *
import re

pd.set_option("display.max_columns", None)
%matplotlib inline

# 4-7 数理最適化（生産計画最適化）

## 4-7-1 プロジェクト起案（事例の概要）

## 4-7-2 業務課題の把握

## 4-7-3 分析方針の決定

- LP: Liner Programming（線形計画法）
- 数理最適化において「解を求める」とは
  - 複数の変数に対してそれらの「値を一意に定める」こと
- 決定変数
  - 解を求める対象となる変数
- 最適解
  - 複数の解の中から何らかの指標に照らして最もよいもの
- 目的関数
  - 最適解に用いる指標

## 4-7-4 データの理解・収集

|No|データ名|内容|
|:-:|:-|:-|
|1|生産可否|各工場でどの製品が生産可能かを表す|
|2|必要リソース|各製品を 1 単位作るために必要なリソース（原料）と必要な量|
|3|リソース上限|各工場で1か月間に使用できるリソース上限|
|4|注文数|仕向国別の各製品の注文数量|
|5|利益|製品 1 単位を販売して得られる利益額（仕向国別）|

### 4-7-5 データの加工

#### (0)定式化の準備

$$
\begin{align*}

\text{生産可否}& \qquad a_{ij} = \begin{cases}
                                    1 \dots \text{工場 $i$ で製品 $j$ を生産可能} \\
                                    0 \dots \text{otherwise}
                                \end{cases}
\newline
\newline
\text{必要リソース}& \qquad r_{jk} (\ge 0): \text{製品 $j$ に対するリソース $k$ の必要量}
\newline
\newline
\text{リソース上限}& \qquad u_{ik} (\ge 0): \text{工場 $i$ のリソース $k$ の上限}
\newline
\newline
\text{受注}& \qquad d_{jl} (\ge 0): \text{製品 $j$ の仕向国 $l$ の注文数}
\newline
\newline
\text{国別利益}& \qquad p_{jl} (\ge 0): \text{製品 $j$ の仕向国 $l$ における利益}
\newline
\newline
\newline
\qquad I: \quad &\text{工場の集合} \quad (0, 1, \dots, i-1)
\newline
\qquad J: \quad &\text{製品の集合} \quad (0, 1, \dots, j-1)
\newline
\qquad K: \quad &\text{リソースの集合} \quad (0, 1, \dots, k-1)
\newline
\qquad L: \quad &\text{仕向国（出荷先）の集合} \quad (0, 1, \dots, l-1)

\end{align*}
$$

##### 入力データの読み込み

In [6]:
DATA_PATH = "../../support/src1300/src_ch04/4.7_生産計画最適化/data/"

a_d = pd.read_csv(
    os.path.join(DATA_PATH, "生産可否.csv"), header=0, index_col=0
)
u_d = pd.read_csv(
    os.path.join(DATA_PATH, "リソース上限.csv"), header=0, index_col=0
)
r_d = pd.read_csv(
    os.path.join(DATA_PATH, "必要リソース.csv"), header=0, index_col=0
)
p_d = pd.read_csv(
    os.path.join(DATA_PATH, "利益.csv"), header=0, index_col=0
)
d_d = pd.read_csv(
    os.path.join(DATA_PATH, "注文数.csv"), header=0, index_col=0
)
a_d = a_d.T

In [9]:
# 生産可否
print(a_d.shape)
display(a_d)

(3, 1000)


,製品1,製品2,製品3,製品4,製品5,製品6,製品7,製品8,製品9,製品10,製品11,製品12,製品13,製品14,製品15,製品16,製品17,製品18,製品19,製品20,製品21,製品22,製品23,製品24,製品25,製品26,製品27,製品28,製品29,製品30,製品31,製品32,製品33,製品34,製品35,製品36,製品37,製品38,製品39,製品40,製品41,製品42,製品43,製品44,製品45,製品46,製品47,製品48,製品49,製品50,製品51,製品52,製品53,製品54,製品55,製品56,製品57,製品58,製品59,製品60,製品61,製品62,製品63,製品64,製品65,製品66,製品67,製品68,製品69,製品70,製品71,製品72,製品73,製品74,製品75,製品76,製品77,製品78,製品79,製品80,製品81,製品82,製品83,製品84,製品85,製品86,製品87,製品88,製品89,製品90,製品91,製品92,製品93,製品94,製品95,製品96,製品97,製品98,製品99,製品100,製品101,製品102,製品103,製品104,製品105,製品106,製品107,製品108,製品109,製品110,製品111,製品112,製品113,製品114,製品115,製品116,製品117,製品118,製品119,製品120,製品121,製品122,製品123,製品124,製品125,製品126,製品127,製品128,製品129,製品130,製品131,製品132,製品133,製品134,製品135,製品136,製品137,製品138,製品139,製品140,製品141,製品142,製品143,製品144,製品145,製品146,製品147,製品148,製品149,製品150,製品151,製品152,製品153,製品154,製品155,製品156,製品157,製品158,製品159,製品160,製品161,製品162,製品163,製品164,製品165,製品166,製品167,製品168,製品169,製品170,製品171,製品172,製品173,製品174,製品175,製品176,製品177,製品178,製品179,製品180,製品181,製品182,製品183,製品184,製品185,製品186,製品187,製品188,製品189,製品190,製品191,製品192,製品193,製品194,製品195,製品196,製品197,製品198,製品199,製品200,製品201,製品202,製品203,製品204,製品205,製品206,製品207,製品208,製品209,製品210,製品211,製品212,製品213,製品214,製品215,製品216,製品217,製品218,製品219,製品220,製品221,製品222,製品223,製品224,製品225,製品226,製品227,製品228,製品229,製品230,製品231,製品232,製品233,製品234,製品235,製品236,製品237,製品238,製品239,製品240,製品241,製品242,製品243,製品244,製品245,製品246,製品247,製品248,製品249,製品250,製品251,製品252,製品253,製品254,製品255,製品256,製品257,製品258,製品259,製品260,製品261,製品262,製品263,製品264,製品265,製品266,製品267,製品268,製品269,製品270,製品271,製品272,製品273,製品274,製品275,製品276,製品277,製品278,製品279,製品280,製品281,製品282,製品283,製品284,製品285,製品286,製品287,製品288,製品289,製品290,製品291,製品292,製品293,製品294,製品295,製品296,製品297,製品298,製品299,製品300,製品301,製品302,製品303,製品304,製品305,製品306,製品307,製品308,製品309,製品310,製品311,製品312,製品313,製品314,製品315,製品316,製品317,製品318,製品319,製品320,製品321,製品322,製品323,製品324,製品325,製品326,製品327,製品328,製品329,製品330,製品331,製品332,製品333,製品334,製品335,製品336,製品337,製品338,製品339,製品340,製品341,製品342,製品343,製品344,製品345,製品346,製品347,製品348,製品349,製品350,製品351,製品352,製品353,製品354,製品355,製品356,製品357,製品358,製品359,製品360,製品361,製品362,製品363,製品364,製品365,製品366,製品367,製品368,製品369,製品370,製品371,製品372,製品373,製品374,製品375,製品376,製品377,製品378,製品379,製品380,製品381,製品382,製品383,製品384,製品385,製品386,製品387,製品388,製品389,製品390,製品391,製品392,製品393,製品394,製品395,製品396,製品397,製品398,製品399,製品400,製品401,製品402,製品403,製品404,製品405,製品406,製品407,製品408,製品409,製品410,製品411,製品412,製品413,製品414,製品415,製品416,製品417,製品418,製品419,製品420,製品421,製品422,製品423,製品424,製品425,製品426,製品427,製品428,製品429,製品430,製品431,製品432,製品433,製品434,製品435,製品436,製品437,製品438,製品439,製品440,製品441,製品442,製品443,製品444,製品445,製品446,製品447,製品448,製品449,製品450,製品451,製品452,製品453,製品454,製品455,製品456,製品457,製品458,製品459,製品460,製品461,製品462,製品463,製品464,製品465,製品466,製品467,製品468,製品469,製品470,製品471,製品472,製品473,製品474,製品475,製品476,製品477,製品478,製品479,製品480,製品481,製品482,製品483,製品484,製品485,製品486,製品487,製品488,製品489,製品490,製品491,製品492,製品493,製品494,製品495,製品496,製品497,製品498,製品499,製品500,製品501,製品502,製品503,製品504,製品505,製品506,製品507,製品508,製品509,製品510,製品511,製品512,製品513,製品514,製品515,製品516,製品517,製品518,製品519,製品520,製品521,製品522,製品523,製品524,製品525,製品526,製品527,製品528,製品529,製品530,製品531,製品532,製品533,製品534,製品535,製品536,製品537,製品538,製品539,製品540,製品541,製品542,製品543,製品544,製品545,製品546,製品547,製品548,製品549,製品550,製品551,製品552,製品553,製品554,製品555,製品556,製品557,製品558,製品559,製品560,製品561,製品562,製品563,製品564,製品565,製品566,製品567,製品568,製品569,製品570,製品571,製品572,製品573,製品574,製品575,製品576,製品577,製品578,製品579,製品580,製品581,製品582,製品583,製品584,製品585,製品586,製品587,製品588,製品589,製品590,製品591,製品592,製品593,製品594,製品595,製品596,製品597,製品598,製品599,製品600,製品601,製品602,製品603,製品604,製品605,製品606,製品607,製品608,製品609,製品610,製品611,製品612,製品613,製品614,製品615,製品616,製品617,製品618,製品619,製品620,製品621,製品622,製品623,製品624,製品625,製品626,製品627,製品628,製品629,製品630,製品631,製品632,製品633,製品634,製品635,製品636,製品637,製品638,製品639,製品640,製品641,製品642,製品643,製品644,製品645,製品646,製品647,製品648,製品649,製品650,製品651,製品652,製品653,製品654,製品655,製品656,製品657,製品658,製品659,製品660,製品661,製品662,製品663,製品664,製品665,製品666,製品667,製品668,製品669,製品670,製品671,製品672,製品673,製品674,製品675,製品676,製品677,製品678,製品679,製品680,製品681,製品682,製品683,製品684,製品6

In [10]:
# リソース上限
print(u_d.shape)
display(u_d)

(3, 12)


,原料1,原料2,原料3,原料4,原料5,原料6,原料7,原料8,原料9,原料10,原料11,原料12
工場X,46205300,46039300,48559750,46034300,47810310,46265760,48303490,46041290,45930390,44539950,47039400,48200000
工場Y,39506400,37453260,30259030,38039300,38305090,39739020,38029400,39305050,35930200,37311030,38400000,37600000
工場Z,13202100,15882600,13227610,15093090,14052510,17049500,13446930,15555640,16443620,15570910,16800000,15600000


In [11]:
# 必要リソース
print(r_d.shape)
display(r_d)

(1000, 12)


,原料1,原料2,原料3,原料4,原料5,原料6,原料7,原料8,原料9,原料10,原料11,原料12
製品1,5.829631,2.074566,4.609442,1.692674,4.794378,4.483725,3.738975,3.344891,3.847448,1.637651,4.889495,4.554688
製品2,3.325596,5.266711,3.513287,3.260987,2.632716,5.141677,4.858524,1.816724,5.097819,2.197664,5.645578,3.570298
製品3,4.286877,3.375680,1.568908,1.790436,2.619886,2.977039,4.301817,2.160792,4.243282,5.488517,4.309042,3.420488
製品4,3.014178,3.932501,5.525522,3.024910,1.667651,3.478191,2.318724,4.066283,4.669278,5.819366,2.822942,1.856178
製品5,3.579886,3.454544,2.818869,4.506821,2.714961,3.110283,4.037710,5.472291,3.244940,5.045309,5.628787,3.528849
...,...,...,...,...,...,...,...,...,...,...,...,...
製品996,3.440771,2.026804,2.555453,1.603293,4.651930,3.243334,5.366599,5.256837,5.998628,3.026882,5.775984,4.929483
製品997,2.182108,2.771958,2.684994,1.663825,3.608071,3.469408,5.169211,3.719946,2.104106,3.735266,3.784345,5.920536
製品998,2.643254,5.718500,1.709486,3.458722,2.163321,2.144578,2.588322,2.025917,1.645177,2.964419,5.770552,5.314389
製品999,4.371847,2.319356,5.273179,2.332740,3.860351,2.058853,2.653568,2.237793,2.492002,4.178498,4.266870,4.392445


In [12]:
# 国別利益
print(p_d.shape)
display(p_d)

(1000, 10)


,A国,B国,C国,D国,E国,F国,G国,H国,I国,J国
製品1,72,111,203,168,263,110,186,88,86,231
製品2,80,225,70,239,99,109,199,161,177,198
製品3,150,234,212,201,99,254,244,86,266,293
製品4,273,295,146,90,259,188,226,182,192,237
製品5,199,84,104,239,287,118,262,217,297,58
...,...,...,...,...,...,...,...,...,...,...
製品996,93,88,198,89,102,154,261,93,187,60
製品997,162,80,290,282,285,239,225,271,298,98
製品998,280,249,118,215,118,54,56,90,265,241
製品999,58,152,199,162,130,210,294,284,84,58


In [13]:
# 注文数
print(d_d.shape)
display(d_d)

(1000, 10)


,A国,B国,C国,D国,E国,F国,G国,H国,I国,J国
製品1,2093,2062,3169,177,2447,335,1398,302,2967,1627
製品2,1299,1297,525,745,3255,4969,1533,1288,4555,877
製品3,4947,2672,3662,266,4966,2374,4858,4666,4192,422
製品4,751,3246,4142,2357,2363,4688,4545,714,2950,4728
製品5,386,2951,4506,1682,261,1634,2252,4820,4623,959
...,...,...,...,...,...,...,...,...,...,...
製品996,2133,3956,1065,547,2816,1270,2562,1098,573,3400
製品997,4562,1680,1757,4011,2493,2644,2591,2320,2981,846
製品998,3181,1902,984,2808,4885,4858,375,819,1442,3180
製品999,4975,2785,1494,781,4022,2702,2141,1664,318,675


##### 定式化: 集合の定義

In [14]:
I = range(len(a_d))          # 工場の集合
J = range(len(a_d.columns))  # 製品の集合
K = range(len(r_d.columns))  # リソースの集合
L = range(len(p_d.columns))  # 仕向け国の集合

##### (1)決定変数の定義

- 各仕向国からの注文量に応じて月々の生産量を柔軟に決定したい
- バックオーダーをできるだけ少なくしたい
  - 「バックオーダー」=「注文数」-「出荷量」

$$
\begin{align}

& x_{ij} (\ge 0): \text{工場 $i$ の製品 $j$ の生産量}
\newline
& y_{jl} (\ge 0): \text{製品 $j$ の仕向国 $l$ への出荷量}
\newline
& z_{jl} (\ge 0): \text{製品 $j$ の仕向国 $l$ のバックオーダー数}

\end{align}
$$

In [16]:
# x：工場の各製品の生産量
x = { (i, j) : LpVariable('x%d_%d'%(i, j), lowBound=0) for i in I for j in J }

# y：製品の仕向け国への出荷量
y = { (j, l) : LpVariable('y%d_%d'%(j, l), lowBound=0) for j in J for l in L }

# z：製品の各仕向け国のバックオーダー数
z = { (j, l) : LpVariable('z%d_%d'%(j, l), lowBound=0) for j in J for l in L }

##### (2)制約条件

生産量 $x$ およびバックオーダー数 $z$ を最適化するにあたって最低限守らなければならない制約条件

- `C-1` リソースの上限
  - 各原料について、それらの使用量の合計は工場ごとに定められた上限を超えてはならない
  - $\displaystyle\sum_{j=0}^{j-1} r_{jk} x_{ij} \le u_{ik} \qquad \forall i \in I, \forall k \in K$
- `C-2` 各工場の製品生産可否
  - 各製品について、生産不可の工場の生産量は 0
  - $\displaystyle x_{ij} = 0 \qquad \forall i \in I, \forall j \in J, a_{ij} = 0$
- `C-3` 生産量と出荷量の関係
  - 各製品について、全工場の生産量の合計は、全仕向国の出荷量の合計以上でなければならない
  - $\displaystyle\sum_{i=0}^{i-1} x_{ij} \le \sum_{l=0}^{l-1} y_{jl} \qquad \forall j \in J$
- `C-4` 出荷量とバックオーダー数の関係
  - バックオーダー数は注文数から出荷量を引いた数である
  - $\displaystyle z_{jl} = d_{jl} - y_{jl} \qquad \forall l \in L$

In [15]:
# C-1: リソース量の上限
def C_1(m):
    for i in I:
        for k in K:
            m += lpDot(
                r_d.iloc[:, k], (x[i, j] for j in J)
            ) <= u_d.iloc[i, k], 'C-1:%s_%d_%d'%(u_d.columns.values[k], i, k)


# C-2: 各工場の製品生産可否
def C_2(m) :
    for i in I:
        for j in J:
            if a_d.iloc[i,j] == 0:
                m += x[i, j] == 0


# C-3: 生産量と出荷量の関係
def C_3(m) :
    for j in J:
        m += lpSum(x[i, j] for i in I) >= lpSum(y[j, l] for l in L)


# C-4: 出荷量とバックオーダー数の関係
def C_4(m) :
    for j in J:
        for l in L:
            m += z[j, l] == d_d.iloc[j, l] - y[j, l]

##### (3)目的関数

- 目的関数は、解の最適化の度合いを図る指標
- 目的関数をより目的に近づけられる解が、より最適な解ということになる
- `LP`を用いる場合、目的関数は線形式（1次式）で表現しなければならない
- 「利益」を最大化
- 「バックオーダー」を最小化

- 「利益」を最大化
  - 各製品の出荷量に仕向国別の利益をかけ、その合計を取ったものを最大化する

$$

Maximize \quad \displaystyle \sum_{j=0}^{j-1} \sum_{l=0}^{l-1} p_{jl} y_{jl}

$$

- 「バックオーダー」を最小化
  - バックオーダーの中で最大の値を持つものを最小にする
  - 「最大値の最小化」

$$
  Minimize \quad
  \begin{matrix}
    max \\ {\scriptsize j \in J, l \in L}
  \end{matrix}
  \space z_{jl}
$$


- バックオーダーを偏りなく最小化するためには、バックオーダーを数量ではなく比率で扱う必要がある
  - 「バックオーダー率」=「バックオーダー数」/「注文数」

$$
  zz (\ge 0): \quad \text{バックオーダー率の最大値}
$$

In [21]:
# zz(添え字なし)：バックオーダー率の最大値
zz = LpVariable('zz', lowBound=0)

- `C-5` 目的関数定義用の補助制約式
  - $zz \ge \dfrac{1}{d_{jl}} z_{jl} \qquad \forall j \in J, \forall l \in L$

In [22]:
# C-5: 目的関数定義用の補助制約式
def C_5(m) :
    for j in J :
        for l in L :
            #m += zz >= 1/float(d_d.iloc[j,l].sum()*t_d.iloc[0,l]) * z[j, l]
            m += zz >= 1/float(d_d.iloc[j,l].sum()) * z[j, l]

In [23]:
# C-6: 各仕向け国のバックオーダー許容率を守る
def C_6(m) :
    for l in L :
        m += lpSum(z[j, l] for j in J) <= t_d.iloc[0, l] * lpSum(d_d.iloc[:, l])

- min-max 型の目的関数と等価な線形式を用いた目的関数

$$
  minimize \quad zz
$$

- 利益とバックオーダーに関する2つの目的関数の定義ができた
- 重要度を加味するため、これらに重み付けして合計すれば全体の目的関数が得られる

&nbsp;

- 2つの目的関数を統合

$$
\begin{align*}
  Maximize \quad w_1 \displaystyle \sum_{j=0}^{j-1} \sum_{l=0}^{l-1} p_{jl} y_{jl} - w_2 zz &
  \newline
  w_1, w_2 \ge 0 &
\end{align*}
$$

In [24]:
# 利益最大化＋バックオーダー率最小化
def obj_func(m, w1, w2) :
    m += (
        # 利益最大化
        w1 * lpSum(
            lpDot(p_d.iloc[j, :], (y[j, l] for l in L)) for j in J
        )
        # バックオーダー率最小化
        (-1) * (w2 * zz)
    )

In [29]:
for j in J:
    for l in L:
        lpDot(p_d[j, :], y[j, l])


InvalidIndexError: (0, slice(None, None, None))

##### (4)最適化実行結果

- 目的関数
  - 利益額は比較的大きな値
  - バックオーダー率は1以下の小さな値
  - $w_1 = 1.0, w_2 = 10^5(100000)$ として実行

In [27]:
#最適化実行

# ソルバ宣言
# 最大化
model = LpProblem(sense=LpMaximize)

# 制約式
C_1(model)
C_2(model)
C_3(model)
C_4(model)
C_5(model)

# 目的関数
obj_func(model, 1.0, 10 ** 5)

# 実行
print(model.solve())

TypeError: 'LpAffineExpression' object is not callable